In [2]:
import os

import openai
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]


client = OpenAI()

file = client.files.create(
    file=open("parking_violation.csv", "rb"), purpose="assistants"
)

assistant = client.beta.assistants.create(
    name="Data visualizer",
    description="You are great at creating beautiful data visualizations. You analyze data present in .csv files, understand trends, and come up with data visualizations relevant to those trends. You also share a brief text summary of the trends observed.",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [file.id]}},
)

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Create 3 data visualizations based on the trends in this file.",
            "attachments": [
                {"file_id": file.id, "tools": [{"type": "code_interpreter"}]}
            ],
        }
    ]
)

# Submit the thread to the assistant (as a new run).
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)
print(f"👉 Run Created: {run.id}")


In [8]:
import time

# Wait for run to complete.
while run.status != "completed":
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print(f"🏃 Run Status: {run.status}")
    time.sleep(1)
else:
    print(f"🏁 Run Completed!")

# Get the latest message from the thread.
message_response = client.beta.threads.messages.list(thread_id=thread.id)
messages = message_response.data


🏁 Run Completed!


In [34]:
message_response.data[1].content

[ImageFileContentBlock(image_file=ImageFile(file_id='file-6N2jeZppTDhIi54AbIqAPAGa', detail=None), type='image_file'),
 TextContentBlock(text=Text(annotations=[], value="The second visualization shows the top 10 vehicle makes with the most violations, where Honda, Nissan, and Toyota lead the list.\n\nFinally, let's create the third visualization, which will focus on the distribution of violation codes."), type='text')]

In [26]:
message_response.content

AttributeError: 'SyncCursorPage[Message]' object has no attribute 'content'

In [23]:
# Print the latest message.
latest_message = messages[4]
print(latest_message)
print(f"💬 Response: {latest_message.content[0].text}")


Message(id='msg_g9811lJxKkb47GtJYBtVLiEQ', assistant_id=None, attachments=[Attachment(file_id='file-4zaBhPXfUovlBFYTt9iVD08K', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create 3 data visualizations based on the trends in this file.'), type='text')], created_at=1718007540, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_35am3TudXzjvJaMNlOSHJCHi')
💬 Response: Text(annotations=[], value='Create 3 data visualizations based on the trends in this file.')


In [25]:
content_block = messages.content[0]

AttributeError: 'list' object has no attribute 'content'